In [9]:
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

#### Step 2: Loading the Dataset

In [50]:
# Transforms images to a PyTorch Tensor
tensor_transform = transforms.ToTensor()
  
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data", # The location of the downlode file
    train=True,
    download=True,
    transform=tensor_transform,
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=tensor_transform,
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(dataset = training_data, 
                              batch_size = batch_size,
                              shuffle = True)
test_dataloader = DataLoader(dataset = test_data, 
                             batch_size=batch_size,
                             shuffle = True)

# for X, y in test_dataloader:
#     print(f"Shape of X [Batch size, Channel, Height, Width]: {X.shape}")
#     print(f"Shape of y: {y.shape} {y.dtype}")
#     break

### Create Autoencoder Class

In [51]:
# Creating a PyTorch class
# 28*28 ==> 9 ==> 28*28
class AE(torch.nn.Module):
    def __init__(self):
        super(AE,self).__init__()
        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 400 ==> 100
        self.encoder = torch.nn.Sequential(
            # torch.nn.Linear(400, 300),
            # torch.nn.ReLU(),
            # torch.nn.Linear(300, 200),
            # torch.nn.ReLU(),
            # torch.nn.Linear(200, 100),
            # torch.nn.ReLU(),
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU()
        )
          
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 100 ==> 400
        self.decoder = torch.nn.Sequential(
            # torch.nn.Linear(100, 200),
            # torch.nn.ReLU(),
            # torch.nn.Linear(200, 300),
            # torch.nn.ReLU(),
            # torch.nn.Linear(300, 400),
            # torch.nn.Sigmoid()
            torch.nn.Linear(32, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28 * 28),
            torch.nn.Sigmoid()
        )
  
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded, 

### Initializing models

In [52]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Model Initialization
model = AE().to(device)
  
# Validation using MSE Loss function
loss_fn = torch.nn.MSELoss()
  
# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(),
                             lr = 1e-2,
                             weight_decay = 1e-8)

model

Using cpu device


AE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=784, bias=True)
    (5): Sigmoid()
  )
)

### Define training function 

In [53]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() #Tell the model you are going to train so the weights will be updated
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        print(pred.shape)

        # Backpropagation
        optimizer.zero_grad() # to reset the gradients of model parameters.
        loss.backward() # PyTorch deposits the gradients of the loss w.r.t. each parameter.
        optimizer.step() # to adjust the parameters by the gradients collected in the backward pass.

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() #Tell the model you are going to test so the weights will not be updated
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) # to is one of the operations(methods) in tensor object
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

###  Start training and validating

In [54]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1792x28 and 784x128)

###  Input/Reconstructed Input to/from Autoencoder

In [ ]:
for i, item in enumerate(image):
    
  # Reshape the array for plotting
  item = item.reshape(-1, 28, 28)
  plt.imshow(item[0])
  
for i, item in enumerate(reconstructed):
  item = item.reshape(-1, 28, 28)
  plt.imshow(item[0])